In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import glob

path_to_data = glob.glob('/home/bdc2/aodhan/ROM_SAF/collocations/*_ERA_5_colocated_occultations.npy')
collocation_arrays = [np.load(path_to_data[i], allow_pickle=True) for i in range(0, len(path_to_data))]
collocation_concatenation = np.concatenate(collocation_arrays, axis=0)


In [3]:
collocation_concatenation[1000000]

array([5, 8, '2008', 61.0, 331.75, -60.18926783152813,
       masked_array(data=[-39.52181850510942, -49.44632907725571,
                   -64.83428170687, -68.51977634904634,
                   -66.77422888665293, -68.84782587214116,
                   -66.38947944598618, -63.08265925330818,
                   -62.44276018356766, -61.440386640777945,
                   -58.71879059732464, -57.64351716051385,
                   -57.48961738424714, -56.83756833216981,
                   -57.53011732536996, -56.99551810254877,
                   -54.79434630252368, -50.179378011578706,
                   -44.489136283822916],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False],
       fill_value=1e+20)], dtype=object)

(11202775, 7)


In [7]:
np.shape(collocation_concatenation)

collocation_concatenation_thing = collocation_concatenation[:,1]

In [8]:
np.unique(collocation_concatenation_thing)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23], dtype=object)